In [13]:
# -*- coding: utf-8 -*-
%matplotlib inline
import re, jieba
import gensim
import difflib
import numpy as np
import random
import opencc
from smart_open import smart_open
import matplotlib.pyplot as plt
import subprocess

ANS = ['a', 'b', 'c', 'd', 'e']

def simple_preprocess(content):
    content = content.strip().replace('︽⊙＿⊙︽', '龐燮傍謝')
    wlist = list(jieba.cut(content))
    qidx = []
    i = 0
    for w in wlist:
        if w == '龐燮傍謝':
            wlist[i] = '*'
            qidx.append(i)
        i += 1
    return (wlist, qidx)

def build_estimate_samples(wlist, qidx):
    temp = wlist[:]
    est_sen = []
    sen_len = len(wlist)
    for i in qidx:
        head = max(i - 10, 0)
        tail = min(i + 10, sen_len)
        est_sen.append(wlist[head : i] + wlist[i + 1 : tail])
    return est_sen

def estimate_ans(est_sen_list, options):
    n_sen = float(len(est_sen_list))
    option_vec_idx = []
    for w in options:
        if w in model: option_vec_idx.append(model.vocab[w].index)
        else: 
            option_vec_idx.append(-1)
            return -1
    score = [0., 0., 0., 0., 0.]
    for wlist in est_sen_list:
        arr = np.zeros(100)
        for w in wlist:
            if w in model and w != u'*': arr += model[w]
        for i in range(5):
            if option_vec_idx[i] >= 0: score[i] += np.dot(arr, syn1neg[option_vec_idx[i]])
    
    for i in range(5):
        score[i] /= n_sen
    
    return ANS[score.index(max(score))]
#     np.dot(arr, syn1neg[idx])




In [2]:
# jieba.add_word('龐燮傍謝', freq=10, tag='xx')
# jieba.load_userdict('data/zhwiki-cn-clean')
# path = 'w2v-experiment/model/'
# model = gensim.models.Word2Vec.load_word2vec_format(path + 'syn0.bin', binary = True)
# vocab_size, vector_size = model.syn0.shape
# syn1neg = np.zeros((vocab_size, vector_size), dtype=np.float32)
# binary_len = np.dtype(np.float32).itemsize * vector_size
# with smart_open(path + 'syn1neg.bin') as fin:
#     for i in range(vocab_size):
#         weights = np.fromstring(fin.read(binary_len), dtype=np.float32)
#         syn1neg[i] = weights

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/4_/b_9vfbwx41g4t48jrzr0r4vm0000gn/T/jieba.cache
Loading model cost 1.255 seconds.
Prefix dict has been built succesfully.


In [14]:
right, wrong = 0, 0
wrong_sen = []
with open('question_official/hackathon_1000.tsv', 'rb') as f:
    for i in range(200): next(f)
    for line in f:
        s = opencc.convert(line.decode('utf-8'), config = '/usr/share/opencc/tw2s.json')
#         print(repr(s))
        parse = s.split('\t')
#         print(parse)
        no = int(parse[0])
        content = parse[1].strip()
        ans = parse[2]
        options = parse[3 : 8]
        ans_ref = parse[8]
        url = parse[9]
        level = parse[10]
#         print(content)
#         print(options, ans_ref)
        wlist, qidx = simple_preprocess(content)
        est_sen = build_estimate_samples(wlist, qidx)
        pred = estimate_ans(est_sen, options)
        if pred == -1: continue
        if pred == ans: right += 1
        else: 
            wrong += 1
            wrong_sen.append((content, options[ANS.index(pred)], ans_ref))

print(float(right)/float(right+wrong))
#  (['脸上', '比较', '油', '的', '时候', '我', '都', '会', '习惯', '让', '*', '多', '停留', '个', '10', 
#    '秒左右', '，', '会', '比', '马上', '就', '洗掉', '清洁', '得', '更', '干净', '一点', '唷'], [10])
      

0.7604355716878403


In [101]:
wrong_sen

[('看到台中路况是好天气，推测苗栗应该会不差。无奈开到︽⊙＿⊙︽交流道仍是阴天，反正天气都比台北好就对了', '大船', '苗栗公馆'),
 ('心得：跟bb霜不同，︽⊙＿⊙︽不以遮瑕力着称，而是用上的保湿感及明亮度会明显的提升，所以没有分色号，几乎都是淡肤色质地，skii的︽⊙＿⊙︽我觉得用起来感觉很不错，保湿度自然不在话下，虽然说遮瑕力对我这痘痘肌人来说还是偏弱，但如果在我肤况好的时候用，蛮锦上添花的，明亮度很不错，几乎可以上这瓶就出门哦',
  '里的',
  'cc霜'),
 ('建于1906年日治时期的乙等官舍，1926年拨供︽⊙＿⊙︽农林学校校长居住宿舍\xa0，2004年重修', '机具', '宜兰'),
 ('angel key无瑕美腿︽⊙＿⊙︽清爽不黏腻\xa0\xa0 夏天很容易流汗\xa0 相信大家都很讨厌身体黏黏的', '字眼', 'cc霜'),
 ('每当外地的朋友来到台中，为了尽地主之道的我们，总是只能祭出逢甲夜市、︽⊙＿⊙︽，或是东海艺术街，再不然就是新社花海、薰衣草花园这一类的景',
  '皇天',
  '一中商圈'),
 ('这是由︽⊙＿⊙︽国小附设幼儿园的小朋友画的唷！来自五位艺术家的原创发想，与小孩的创作实验让整个空间呈现出一种动物狂观的奇景，是不是让整个空间更丰富热闹了起来呢',
  '国玩',
  '中正'),
 ('从火车站后站出口的前锋路只看得到︽⊙＿⊙︽的招牌，所以有些台南人以为这栋圆形的高楼是商场＋办公大楼，其实走到大学路西段，就可以看到饭店的招牌了',
  '理性',
  '大远百'),
 ('︽⊙＿⊙︽老屋改建为日租套房的案例不少，不过的确是很受旅人们的欢迎，似乎已成了一种体验︽⊙＿⊙︽风情的选项，也是主人分享自己创意风格的好地方',
  '里的',
  '台南'),
 ('︽⊙＿⊙︽教会在这里传教已经超过百年，应该算是花东纵谷中历史最悠久的教会了。教会的位置并不难找，就在︽⊙＿⊙︽国中旁边，为何说这个教会的建筑很奇特，是因为它看起了像是一个欧式的钟塔，上面的水泥不知是否年代久远有着颗粒斑驳的感觉',
  '译名',
  '富里'),
 ('︽⊙＿⊙︽的幽静之美不用多说了，此湖三面环山，面积约17公顷，湖面辽阔，是宜兰五大名湖中面积最大者', '文人', '龙潭湖'),
 ('其中访问到一个日本老板，他这样说：

In [12]:
model.most_similar(['松烟文创'])

[('松山文创', 0.7016356587409973),
 ('南港软体', 0.6479178667068481),
 ('旁Wiz', 0.6130987405776978),
 ('Waffogato', 0.5951836705207825),
 ('@内科', 0.5815931558609009),
 ('内湖', 0.5726386904716492),
 ('南港', 0.5714350938796997),
 ('@捷运港墘站', 0.5695842504501343),
 ('榆小', 0.5559353828430176),
 ('松烟', 0.5554336309432983)]

In [136]:
word_dict = ' '.join(list(model.vocab))

In [149]:
for match in re.findall(r'\s(\S*文创\S*)\s', word_dict): 
    print(match)
    

文创化
成文创
文创工作者
文创
中文创
美地文创
拥恒文创
十鼓文创
文创头围
文创园区
文创产业
台北文创
青青文创
谷物文创
丰味文创
文创舘
灿星文创
文创业者
阿之宝文创
莎贝莉娜文创
华山文创产业
松山文创
三莺文创
华山文创
起初文创
经贸文创
范特喜文创
甘乐文创
文创聚落
文创生活
著文创
学学文创
文创艺品
花莲文创产业
猫头鹰文创
台北文创大楼
头围文创
图文创
文创商品
松烟文创
文创产业园区


In [150]:
a = '松烟文创'
print(HanziConv.toTraditional(a))


鬆煙文創
